In [1]:
from canvasdata import CanvasData
from coursedata import CourseData
from student_changepoint import StudentChangepoint
from datetime import datetime, timedelta
import pickle
import csv

def process_data():
    """
    Returns
    -------
    course_students: dict
        keys are courses and values are student ids that belong to the course and that are used in the models
    """
    classes = ['./16Fa CHEM 1P', './16S1 PHY 3A',
               './17Fa PUBHLTH 1', './17Fa PUBHLTH 2', './17S1 PHY 3A',
               './17Sp PUBHLTH 1', './17Sp PUBHLTH 2', './17Wi PUBHLTH 1',
               './17Wi PUBHLTH 2', './18S1 CHEM 1C',
               './18Wi BIO SCI 9B']

    #problem with 17S1 PHY 3A: students appear twice in data for some reason 

    course_students = dict()
    for cla in classes:
        print("CLASS", cla)
        duration = 70
        if cla != './2230':
            if "16Fa" in cla:
                first =datetime(2016,9,22,0,0,0)
            elif "16S1" in cla:
                first =datetime(2016,6,20,0,0,0)
                duration = 35
            elif "17Wi" in cla:
                first =datetime(2017,1,9,0,0,0)
            elif "17Fa" in cla:
                first =datetime(2017,9,28,0,0,0)
            elif "17S1" in cla:
                first = datetime(2017,6,26,0,0,0)
                duration = 35
            elif "17Sp" in cla:
                first = datetime(2017,4,3,0,0,0)
            elif "18S1" in cla:
                first = datetime(2018,6,25,0,0,0)
                duration = 35
            elif "18Wi" in cla:
                first = datetime(2018,1,7,0,0,0)
            c = CanvasData(cla, days_limit = duration, first_day = first, depth = 1)         
        else:
            c = CanvasData(cla, depth = 1)

        file = "./training_data/" + cla[2:]
        #c.create_training_data(file, "background")
        #c.create_training_data(file, "week_sessions")
        #c.create_training_data(file, "total_sessions")
        #c.create_training_data(file, "total_clicks")
        #c.create_training_data(file, "weeks")
        #c.create_training_data(file, "weeks_time")
        #c.create_training_data(file, "category_time")
        #c.create_training_data(file, "total_time")
        student_ids = c.create_training_data(file, "categories") 
        course_students[cla[2:]] = student_ids

    return course_students 
    #returned for the purpose of generating long term GPA; won't normally need it for regular training data generation

In [ ]:
def generate_quarters(course_students):
    """
    Determines the long term quarters 
    
    Returns
    -------
    course_students_new: dict
        keys are courses, values are ids associated with each course including ones with a or b appended to the end
    """
    students = dict()
    course_students_new = dict()
    for course in course_students:
        ids = []
        year = course[:2]
        quarter = course[2:4]
        if quarter == "Fa":
            following = [str(int(year)+1) + "Winter", str(int(year)+1) + "Spring", str(int(year)+1) + "Fall"]
        elif quarter == "Wi":
            following = [year + "Spring", year + "Fall", str(int(year)+1)+ "Winter"]
        elif quarter == "Sp":
            following = [year + "Fall", str(int(year)+1) + "Winter", str(int(year)+1) + "Spring"]
        else:
            following = [year + "Fall", str(int(year)+1) + "Winter", str(int(year)+1) + "Spring"]

        for s in course_students[course]:
            if str(s) not in students:
                students[str(s)] = following
                ids.append(str(s))
            elif str(s) + "a" not in students:
                students[str(s) + "a"] = following
                ids.append(str(s) + "a")
            else: 
                students[str(s) + "b"] = following
                ids.append(str(s) + "b")
        course_students_new[course] = ids
    
    with open('./student_data.pickle', 'wb') as handle:
        pickle.dump(students, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
    return course_students_new

In [ ]:
def get_longGPA(ids):
    """
    Gets long term GPA from pickle file and saves them as csv files for individual courses
    """
    with open('./student_gpa.pickle', 'rb') as handle:
        GPA = pickle.load(handle)
        
    for c in ids:
        lines = [["longGPA"]]
        for id in ids[c]:
            lines.append([GPA[id]])
        
        with open("./training_data/" + c + "_longGPA.csv", 'w', newline='') as f:
            writer = csv.writer(f, delimiter=',')
            writer.writerows(lines)

In [ ]:
course_students = process_data()
ids = generate_quarters(course_students)
get_longGPA(ids)